In [1]:
import pandas as pd
import unicodedata

In [2]:
# マスター
df = pd.read_csv('master.csv',
                 usecols=[1, 4, 6, 7, 8, 9, 26, 27, 29],
                 dtype = str).fillna(0).astype({'収容数': int, '回転枚数': int, '読取枚数': int, '発注枚数': int})
df.rename(columns={'かんＳＥＬＦ': 'ad',
                    '仕入先': 'sup_code',
                    '背番号': 'seban',
                    '品番': 'hinban',
                    '収容数': 'num',
                    'ストアアドレス': 'store',
                    '回転枚数': 'k_num',
                    '読取枚数': 'y_num',
                    '発注枚数': 'h_num'}, inplace=True)

df['seban'] = df['seban'].str.strip()
df['hinban'] = df['hinban'].str.strip()
df['store'] = df['store'].str.strip()
df.head()

,ad,sup_code,seban,hinban,num,store,k_num,y_num,h_num
0,02883,01010,10,A1300-10113,10,ｳﾝﾕｵﾄﾜ,14,0,0
1,25245,01010,,A1300-F0H98-00,20,ｳﾝﾕｵﾄﾜ,0,0,0
2,00093,02010,U067,94223-61000-00,1750,03475,1,0,0
3,00102,02010,U068,94223-80600-00,4000,03475,4,0,0
4,00103,02010,U069,94223-80800-00,2500,03475,24,0,4


In [ ]:
# 仕入先マスター
df_s = pd.read_csv('sup.csv',
                 usecols=[0, 1],
                 dtype = str).fillna(0)
df_s.rename(columns={'仕入先': 'sup_code','仕入先名': 'sup_name'}, inplace=True)

df_s['sup_name'] = df_s['sup_name'].str.strip()

df_s.head()

In [2]:
# def add_space_if_fw(text):
#     if unicodedata.east_asian_width(text) in 'FWA':
#         return text + ' '
#     else:
#         return text

In [ ]:
# df = {}
# for i in range(1,3):
#     file_name = f'test{i}.csv'
#     with open(file_name, 'r', encoding='UTF-8') as f:
#         content = f.read()
#     result = ''
#     for ch in content:
#         result += add_space_if_fw(ch)
#     file_name_changed = f'changed_test{i}.csv'
#     with open(file_name_changed, mode='w', encoding='UTF-8') as f:
#         f.write(result)
#     widths = [7, 6]
#     if i == 1:
#         names = ['name', 'code']
#     else:
#         names = ['code', 'text']
#     df[i] = pd.read_fwf(file_name_changed, widths=widths, names=names, encoding='UTF-8')
# for i in df:
#     print(df[i])

In [12]:
import sqlite3

In [13]:
con = sqlite3.connect('test.db')
cur = con.cursor()

In [5]:
# テーブル削除
cur.execute(
    'DROP TABLE IF EXISTS master'
)

In [6]:
# マスター作成
cur.execute(
    '''
    CREATE TABLE master (
        id INTEGER PRIMARY KEY, 
        ad TEXT, 
        sup_code TEXT, 
        seban TEXT, 
        hinban TEXT, 
        num INTEGER, 
        store TEXT, 
        k_num INTEGER, 
        y_num INTEGER, 
        h_num INTEGER)
    '''
)						

In [56]:
# 集欠マスター作成
cur.execute(
    '''
    CREATE TABLE shuketu (
        id INTEGER PRIMARY KEY, 
        ad TEXT, 
        num INTEGER,
        num_all INTEGER, 
        cust_name TEXT, 
        due_date TEXT, 
        tonyu INTEGER,
        inventory INTEGER, 
        afure INTEGER, 
        shuketubi TEXT, 
        bin INTEGER,
        comment TEXT)
    '''
)		

In [6]:
# 仕入先マスター作成
cur.execute(
    '''
    CREATE TABLE sup (
        id INTEGER PRIMARY KEY, 
        sup_code TEXT,
        sup_name TEXT)
    '''
)		

In [7]:
# マスター→DB
df.to_sql('master', con, if_exists='append', index=False)

40048

In [9]:
# 仕入先マスター→DB
df_s.to_sql('sup', con, if_exists='append', index=False)

596

In [24]:
cur.execute(
    "select * from sqlite_master where type='table' and name='master'"
)
tables = cur.fetchall()
tables

[('table',
  'master',
  'master',
  2,
  'CREATE TABLE master (\n        id INTEGER PRIMARY KEY, \n        ad TEXT, \n        sup_code TEXT, \n        seban TEXT, \n        hinban TEXT, \n        num INTEGER, \n        store TEXT, \n        k_num INTEGER, \n        y_num INTEGER, \n        h_num INTEGER)')]

In [ ]:
# データフレームからテーブル作成
# for i in df:
#     df[i].to_sql(f'items{i}', con, if_exists='append', index = False)

#     cur.execute(
#         f'SELECT * FROM items{i}'
#     )
#     print(cur.fetchall())

In [8]:
# 全テーブル検索
cur.execute(
    "SELECT NAME FROM sqlite_master WHERE type='table'"
)
tables = cur.fetchall()
tables

[('sqlite_sequence',), ('shuketu',), ('sup',), ('master',)]

In [15]:
cur.execute(
    "SELECT * FROM master WHERE hinban like '82715%'"
)
tables = cur.fetchall()
tables

[(2271, '28324', '02204', '522', '82715-62070-E0', 60, 'CQ-5-2', 3, 0, 0),
 (2288, '05927', '02204', '480', '82715-11670-00', 100, 'CQ-05-5', 5, 0, 0),
 (2289, '06817', '02204', '481', '82715-11680-00', 100, 'CQ-03-5', 5, 0, 5),
 (2306, '36995', '02204', '529', '82715-55060-00', 100, 'MH01', 4, 0, 0),
 (2317, '40545', '02204', '537', '82715-25F60-00', 160, 'MH01', 7, 0, 0),
 (2318, '40553', '02204', '538', '82715-6AP90-E0', 60, 'MH01', 21, 0, 5),
 (2338, '34761', '02204', '542', '82715-25F80-00', 100, 'MH03', 9, 0, 0),
 (2341, '34766', '02204', '545', '82715-25L50-00', 100, 'MH03', 9, 0, 0),
 (2346, '03521', '02204', '548', '82715-48V20-00', 120, 'MH03', 0, 0, 0),
 (2350, '19859', '02204', '552', '82715-12V80-00', 100, 'MH03', 0, 0, 0),
 (2583, '08729', '02206', '', '82715-6A240-E1', 300, '', 0, 0, 0),
 (3184, '08779', '02206', '1500', '82715-33B61-00', 100, 'Dｼｼﾞｵｷﾊﾞ', 0, 0, 0),
 (3381, '02523', '0220Q', '', '82715-33B61-00', 100, 'Q23-03-8', 0, 0, 0),
 (3523, '00876', '02290', '154',

In [ ]:
cur.execute(
    '''
    INSERT INTO
	    shuketu
    VALUES
        (1,
        '00011',
        1,
        10,
        'aaa',
        '',
        2,
        1,
        1,
        date('now', 'localtime'),
        1,
        '')
    '''
)

In [10]:
con.close()

In [25]:
# モデル設定
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
from sqlalchemy.orm import Session
import datetime
from sqlalchemy import Column, ForeignKey, Integer, String, ForeignKey
from sqlalchemy.orm import relationship

SQLALCHEMY_DATABASE_URL = 'sqlite:///test.db'


engine = create_engine(
  SQLALCHEMY_DATABASE_URL,
  connect_args={'check_same_thread': False}
)

SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)
Base = declarative_base()

class Master(Base):
  __tablename__ = 'master'
  id = Column(Integer, primary_key=True, autoincrement=True, index=True)
  ad = Column(String, unique=True, index=True)
  sup_code = Column(String, index=True)
  seban = Column(String, index=True)
  hinban = Column(String, index=True)
  num = Column(Integer, index=True)
  store = Column(String, index=True)
  k_num = Column(Integer, index=True)
  y_num = Column(Integer, index=True)
  h_num = Column(Integer, index=True)
  shuketu = relationship('Shuketu', backref="master")

class Shuketu(Base):
  __tablename__ = 'shuketu'
  id = Column(Integer, primary_key=True, autoincrement=True, index=True)
  ad = Column(String, ForeignKey('master.ad'), index=True)
  num = Column(Integer, index=True)
  num_all = Column(Integer, index=True)
  cust_name = Column(String, index=True)
  due_date = Column(String, index=True)
  tonyu = Column(Integer, index=True)
  inventory = Column(Integer, index=True)
  afure = Column(Integer, index=True)
  shuketubi = Column(String, index=True)
  bin = Column(Integer, index=True)
  comment = Column(String, index=True)
  

In [8]:
# create
# def create_item(db, args):
#     item = Master(
#         ad=args['ad'], 
#         sup_code=args['sup_code'], 
#         seban=args['seban'], 
#         hinban=args['hinban'], 
#         num=args['num'], 
#         store=args['store'], 
#         k_num=args['k_num'], 
#         y_num=args['y_num'], 
#         h_num=args['h_num'])
#     db.add(item)
#     db.commit()
#     db.refresh(item)
#     return item

def create_item(db, args):

    item = Shuketu(
        ad=args['ad'],
        num=args['num'],
        num_all=args['num_all'],
        cust_name=args['cust_name'],
        due_date=args['due_date'],
        tonyu=args['tonyu'],
        inventory=args['inventory'],
        afure=args['afure'],
        shuketubi=args['shuketubi'],
        bin=args['bin'],
        comment=args['comment'],
    )
    db.add(item)
    db.commit()
    db.refresh(item)
    return item


# args = {}
# args['ad'] = '00001'
# args['sup_code'] = '00100'
# args['seban'] = '010'
# args['hinban'] = '12345-67890-00'
# args['num'] = 100
# args['store'] = 'A01-01-1'
# args['k_num'] = 1
# args['y_num'] = 1 
# args['h_num'] = 1

args = {}
args['ad']= '00010'
args['num']= 1
args['num_all']= 10
args['cust_name']= 'aaa'
args['due_date']= ''
args['tonyu']= 2
args['inventory']= 1
args['afure']= 1
args['shuketubi']= datetime.date.today().isoformat()
args['bin']= 1
args['comment']= ''


db = SessionLocal()
item = create_item(db, args)    
# print(item.id, item.ad, item.sup_code, item.seban, item.hinban, item.num, item.store, item.k_num, item.y_num, item.h_num)
db.close()

In [ ]:
# read
# def read_item(db, ad):
#   items = db.query(Master).filter(Master.ad == ad).all()
#   return items

def read_item(db, ad):
  items = db.query(Shuketu).filter(Shuketu.ad == ad).all()
  return items


ad = '00010'
db = SessionLocal()
items = read_item(db, ad)
for item in items:
    # print(item.id, item.ad, item.sup_code, item.seban, item.hinban, item.num, item.store, item.k_num, item.y_num, item.h_num)
    print(item.id, item.ad, item.num, item.num_all, item.cust_name, item.due_date, item.tonyu, item.inventory, item.afure, item.shuketubi, item.bin, item.comment)
    print(item.master.hinban)
db.close()

In [13]:
# update
# def update_item(db, args):
#     item = db.query(Master).filter(Master.ad == args['ad']).first()
#     item.num = args['num']
#     db.commit()
#     db.refresh(item)
#     return item

def update_item(db, args):
    item = db.query(Shuketu).filter(Shuketu.id == args['id']).first()
    item.num = args['num']
    db.commit()
    db.refresh(item)
    return item

args = {}
args['id'] = 3
args['ad'] = '00001'
args['sup_code'] = '00100'
args['seban'] = '010'
args['hinban'] = '12345-67890-00'
args['num'] = 50000
args['store'] = 'A01-01-1'
args['k_num'] = 1
args['y_num'] = 1 
args['h_num'] = 1

db = SessionLocal()
item = update_item(db, args)
# print(item.id, item.ad, item.sup_code, item.seban, item.hinban, item.num, item.store, item.k_num, item.y_num, item.h_num)
print(item.num)
db.close()

50000


In [15]:
# delete
# def delete_item(db, id):
#     item = db.query(Master).filter(Master.id == id).first()
#     db.delete(item)
#     db.commit()

def delete_item(db, id):
    item = db.query(Shuketu).filter(Shuketu.id == id).first()
    db.delete(item)
    db.commit()

id = 3

db = SessionLocal()
delete_item(db, id)
db.close()